In [1]:
import numpy as np
import time
from datetime import datetime

# --- Classical Chemistry Imports ---
from pyscf import gto, scf, fci

# --- Qiskit Nature Imports (Modern API) ---
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

# --- Qiskit Algorithm Imports (Modern API) ---
from qiskit_algorithms import VQE, AdaptVQE
from qiskit_algorithms.optimizers import COBYLA
from qiskit.primitives import Estimator

# ----- Configuration -----
BASIS_SET = "sto-3g"
BOND_LENGTH = 0.7414  # Angstrom

# ----- Main Calculation Function -----
def main():
    """
    Calculates the ground state energy of H2 using four different methods:
    UHF, FCI, standard VQE, and AdaptVQE.
    """
    print(f"H₂ Ground State Calculation Comparison for Basis: {BASIS_SET}")
    print(f"Started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")

    # Define the molecule's geometry as a string
    atom_str = f"H 0 0 0; H 0 0 {BOND_LENGTH}"

    # Dictionary to store results
    results = {}

    # --- 1. Classical Calculations (UHF and FCI) ---
    print("--- Running Classical Calculations ---")
    start_time = time.time()
    mol = gto.M(atom=atom_str, basis=BASIS_SET, verbose=0)
    
    # UHF Calculation
    mf_uhf = scf.UHF(mol).run()
    results['UHF'] = mf_uhf.e_tot
    # MODIFIED: Removed rounding
    print(f"UHF Energy: {results['UHF']} Hartree")
    
    # FCI Calculation
    # Note: For H2, RHF is sufficient as a starting point for FCI
    mf_rhf = scf.RHF(mol).run()
    results['FCI'] = fci.FCI(mol, mf_rhf.mo_coeff).kernel()[0]
    # MODIFIED: Removed rounding
    print(f"FCI (Exact) Energy: {results['FCI']} Hartree")
    print(f"Classical calculations finished in {time.time() - start_time:.2f}s\n")

    # --- 2. Quantum Calculations (VQE and AdaptVQE) ---
    
    # Set up the chemical problem for Qiskit
    driver = PySCFDriver(atom=atom_str, basis=BASIS_SET)
    problem = driver.run()
    mapper = JordanWignerMapper()

    # Define the UCCSD ansatz, which is used by both VQE methods
    ansatz = UCCSD(
        problem.num_spatial_orbitals,
        problem.num_particles,
        mapper,
        initial_state=HartreeFock(
            problem.num_spatial_orbitals,
            problem.num_particles,
            mapper,
        ),
    )
    optimizer = COBYLA(maxiter=1000)
    

    # Define the VQE instance that both algorithms will use
    vqe_instance = VQE(Estimator(), ansatz, optimizer)
    vqe_instance.initial_point = np.zeros(ansatz.num_parameters)

    # --- Standard VQE Calculation ---
    print("--- Running Standard VQE Calculation ---")
    start_time = time.time()
    solver_vqe = GroundStateEigensolver(mapper, vqe_instance)
    result_vqe = solver_vqe.solve(problem)
    results['VQE'] = result_vqe.total_energies[0]
    # MODIFIED: Removed rounding
    print(f"Standard VQE Energy: {results['VQE']} Hartree")
    print(f"VQE finished in {time.time() - start_time:.2f}s\n")
    
    # --- AdaptVQE Calculation ---
    print("--- Running AdaptVQE Calculation ---")
    start_time = time.time()
    adapt_vqe = AdaptVQE(vqe_instance)
    # This temporary fix is needed for compatibility in some Qiskit versions
    adapt_vqe.supports_aux_operators = lambda: True
    
    solver_adapt = GroundStateEigensolver(mapper, adapt_vqe)
    result_adapt = solver_adapt.solve(problem)
    results['AdaptVQE'] = result_adapt.total_energies[0]
    # MODIFIED: Removed rounding
    print(f"AdaptVQE Energy: {results['AdaptVQE']} Hartree")
    print(f"AdaptVQE finished in {time.time() - start_time:.2f}s\n")

    # --- Final Summary ---
    print("\n" + "="*50)
    print("               Final Energy Summary")
    print("="*50)
    # MODIFIED: Removed all rounding formatters
    print(f"  UHF Energy:          {results['UHF']} Hartree")
    print(f"  Standard VQE Energy:   {results['VQE']} Hartree")
    print(f"  AdaptVQE Energy:       {results['AdaptVQE']} Hartree")
    print(f"  FCI (Exact) Energy:  {results['FCI']} Hartree")
    print("-" * 50)
    # Compare quantum results to the exact FCI energy
    print(f"  Error (VQE vs FCI):    {abs(results['VQE'] - results['FCI'])} Hartree")
    print(f"  Error (AdaptVQE vs FCI): {abs(results['AdaptVQE'] - results['FCI'])} Hartree")
    print("="*50)

if __name__ == "__main__":
    main()

H₂ Ground State Calculation Comparison for Basis: sto-3g
Started at: 2025-09-16 15:40:26

--- Running Classical Calculations ---
UHF Energy: -1.1166843870853413 Hartree
FCI (Exact) Energy: -1.137270174660903 Hartree
Classical calculations finished in 0.36s

--- Running Standard VQE Calculation ---


/tmp/ipykernel_399149/549131400.py:79: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  vqe_instance = VQE(Estimator(), ansatz, optimizer)


Standard VQE Energy: -1.137270173160823 Hartree
VQE finished in 1.84s

--- Running AdaptVQE Calculation ---
AdaptVQE Energy: -1.1372701730186892 Hartree
AdaptVQE finished in 1.02s


               Final Energy Summary
  UHF Energy:          -1.1166843870853413 Hartree
  Standard VQE Energy:   -1.137270173160823 Hartree
  AdaptVQE Energy:       -1.1372701730186892 Hartree
  FCI (Exact) Energy:  -1.137270174660903 Hartree
--------------------------------------------------
  Error (VQE vs FCI):    1.5000800601683295e-09 Hartree
  Error (AdaptVQE vs FCI): 1.642213920405311e-09 Hartree


In [2]:
import numpy as np
import time
from datetime import datetime

# --- Classical Chemistry Imports ---
from pyscf import gto, scf, fci

# --- Qiskit Nature Imports (Modern API) ---
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

# --- Qiskit Algorithm Imports (Modern API) ---
from qiskit_algorithms import VQE  # AdaptVQE removed
from qiskit_algorithms.optimizers import SLSQP
from qiskit.primitives import Estimator

# ----- Configuration -----
BASIS_SET = "3-21g"
BOND_LENGTH = 0.8561
  # Angstrom, standard bond length for H3+

# ----- Main Calculation Function -----
def main():
    """
    Calculates the ground state energy of H3+ using three different methods:
    UHF, FCI, and standard VQE.
    """
    print(f"H₃⁺ Ground State Calculation Comparison for Basis: {BASIS_SET}")
    print(f"Started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")

    # Define the molecule's geometry as an equilateral triangle
    r = BOND_LENGTH / np.sqrt(3)
    atom_str = (
        f"H {r:.4f} 0.0 0.0; "
        f"H {-r/2:.4f} {r * np.sqrt(3)/2:.4f} 0.0; "
        f"H {-r/2:.4f} {-r * np.sqrt(3)/2:.4f} 0.0"
    )

    # Dictionary to store results
    results = {}

    # --- 1. Classical Calculations (UHF and FCI) ---
    print("--- Running Classical Calculations ---")
    start_time = time.time()
    # Note: charge=1 for the H3+ cation
    mol = gto.M(atom=atom_str, basis=BASIS_SET, charge=1, spin=0, verbose=0)
    
    # UHF Calculation
    mf_uhf = scf.UHF(mol).run()
    results['UHF'] = mf_uhf.e_tot
    # MODIFIED: Removed rounding
    print(f"UHF Energy: {results['UHF']} Hartree")
    
    # FCI Calculation
    mf_rhf = scf.RHF(mol).run()
    results['FCI'] = fci.FCI(mol, mf_rhf.mo_coeff).kernel()[0]
    # MODIFIED: Removed rounding
    print(f"FCI (Exact) Energy: {results['FCI']} Hartree")
    print(f"Classical calculations finished in {time.time() - start_time:.2f}s\n")

    # --- 2. Quantum Calculation (VQE) ---
    
    # Set up the chemical problem for Qiskit, specifying charge=1
    driver = PySCFDriver(atom=atom_str, basis=BASIS_SET, charge=1, spin=0)
    problem = driver.run()
    mapper = JordanWignerMapper()

    # Define the UCCSD ansatz
    ansatz = UCCSD(
        problem.num_spatial_orbitals,
        problem.num_particles,
        mapper,
        initial_state=HartreeFock(
            problem.num_spatial_orbitals,
            problem.num_particles,
            mapper,
        ),
    )

    # Define the VQE instance
    vqe_instance = VQE(Estimator(), ansatz, SLSQP())
    vqe_instance.initial_point = np.zeros(ansatz.num_parameters)

    # --- Standard VQE Calculation ---
    print("--- Running Standard VQE Calculation ---")
    start_time = time.time()
    solver_vqe = GroundStateEigensolver(mapper, vqe_instance)
    result_vqe = solver_vqe.solve(problem)
    results['VQE'] = result_vqe.total_energies[0]
    # MODIFIED: Removed rounding
    print(f"Standard VQE Energy: {results['VQE']} Hartree")
    print(f"VQE finished in {time.time() - start_time:.2f}s\n")
    
    # --- Final Summary ---
    print("\n" + "="*50)
    print("          Final Energy Summary")
    print("="*50)
    # MODIFIED: Removed all rounding
    print(f"  UHF Energy:         {results['UHF']} Hartree")
    print(f"  Standard VQE Energy:  {results['VQE']} Hartree")
    print(f"  FCI (Exact) Energy:   {results['FCI']} Hartree")
    print("-" * 50)
    # Compare quantum results to the exact FCI energy
    print(f"  Error (VQE vs FCI):   {abs(results['VQE'] - results['FCI'])} Hartree")
    print("="*50)

if __name__ == "__main__":
    main()

H₃⁺ Ground State Calculation Comparison for Basis: 3-21g
Started at: 2025-09-16 15:45:55

--- Running Classical Calculations ---
UHF Energy: -1.269979193108904 Hartree
FCI (Exact) Energy: -1.2998947445257898 Hartree
Classical calculations finished in 0.21s

--- Running Standard VQE Calculation ---


/tmp/ipykernel_399149/1287379059.py:83: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  vqe_instance = VQE(Estimator(), ansatz, SLSQP())


Standard VQE Energy: -1.2998945516238496 Hartree
VQE finished in 135.75s


          Final Energy Summary
  UHF Energy:         -1.269979193108904 Hartree
  Standard VQE Energy:  -1.2998945516238496 Hartree
  FCI (Exact) Energy:   -1.2998947445257898 Hartree
--------------------------------------------------
  Error (VQE vs FCI):   1.9290194019916385e-07 Hartree


In [3]:
import numpy as np
import time
from datetime import datetime

# --- Classical Chemistry Imports ---
from pyscf import gto, scf, fci

# --- Qiskit Nature Imports (Modern API) ---
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

# --- Qiskit Algorithm Imports (Modern API) ---
from qiskit_algorithms import VQE  # AdaptVQE removed
from qiskit_algorithms.optimizers import COBYLA
from qiskit.primitives import Estimator

# ----- Configuration -----
BASIS_SET = "sto-3g"
BOND_LENGTH = 0.97  # Angstrom, equilibrium bond length for OH-

# ----- Main Calculation Function -----
def main():
    """
    Calculates the ground state energy of OH- using three different methods:
    UHF, FCI, and standard VQE.
    """
    print(f"OH⁻ Ground State Calculation Comparison for Basis: {BASIS_SET}")
    print(f"Started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")

    # Define the molecule's geometry as a string
    atom_str = f"O 0 0 0; H 0 0 {BOND_LENGTH}"

    # Dictionary to store results
    results = {}

    # --- 1. Classical Calculations (UHF and FCI) ---
    print("--- Running Classical Calculations ---")
    start_time = time.time()
    # Note: charge=-1 and spin=0 for the hydroxide anion
    mol = gto.M(atom=atom_str, basis=BASIS_SET, charge=-1, spin=0, verbose=0)
    
    # UHF Calculation
    mf_uhf = scf.UHF(mol).run()
    results['UHF'] = mf_uhf.e_tot
    # MODIFIED: Removed rounding
    print(f"UHF Energy: {results['UHF']} Hartree")
    
    # FCI Calculation
    mf_rhf = scf.RHF(mol).run()
    results['FCI'] = fci.FCI(mol, mf_rhf.mo_coeff).kernel()[0]
    # MODIFIED: Removed rounding
    print(f"FCI (Exact) Energy: {results['FCI']} Hartree")
    print(f"Classical calculations finished in {time.time() - start_time:.2f}s\n")

    # --- 2. Quantum Calculation (VQE) ---
    
    # Set up the chemical problem for Qiskit, specifying charge=-1
    driver = PySCFDriver(atom=atom_str, basis=BASIS_SET, charge=-1, spin=0)
    problem = driver.run()
    mapper = JordanWignerMapper()

    # Define the UCCSD ansatz
    ansatz = UCCSD(
        problem.num_spatial_orbitals,
        problem.num_particles,
        mapper,
        initial_state=HartreeFock(
            problem.num_spatial_orbitals,
            problem.num_particles,
            mapper,
        ),
    )

    # Define the VQE instance
    vqe_instance = VQE(Estimator(), ansatz, COBYLA(maxiter=1500))
    vqe_instance.initial_point = np.zeros(ansatz.num_parameters)

    # --- Standard VQE Calculation ---
    print("--- Running Standard VQE Calculation ---")
    start_time = time.time()
    solver_vqe = GroundStateEigensolver(mapper, vqe_instance)
    result_vqe = solver_vqe.solve(problem)
    results['VQE'] = result_vqe.total_energies[0]
    # MODIFIED: Removed rounding
    print(f"Standard VQE Energy: {results['VQE']} Hartree")
    print(f"VQE finished in {time.time() - start_time:.2f}s\n")
    
    # --- Final Summary ---
    print("\n" + "="*50)
    print("               Final Energy Summary")
    print("="*50)
    # MODIFIED: Removed all rounding
    print(f"  UHF Energy:          {results['UHF']} Hartree")
    print(f"  Standard VQE Energy:   {results['VQE']} Hartree")
    print(f"  FCI (Exact) Energy:  {results['FCI']} Hartree")
    print("-" * 50)
    # Compare quantum results to the exact FCI energy
    print(f"  Error (VQE vs FCI):    {abs(results['VQE'] - results['FCI'])} Hartree")
    print("="*50)

if __name__ == "__main__":
    main()

OH⁻ Ground State Calculation Comparison for Basis: sto-3g
Started at: 2025-09-16 15:48:47

--- Running Classical Calculations ---
UHF Energy: -74.05739918919758 Hartree
FCI (Exact) Energy: -74.0817233596675 Hartree
Classical calculations finished in 0.27s

--- Running Standard VQE Calculation ---


/tmp/ipykernel_399149/307739788.py:77: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  vqe_instance = VQE(Estimator(), ansatz, COBYLA(maxiter=1500))


Standard VQE Energy: -74.08172177081973 Hartree
VQE finished in 766.55s


               Final Energy Summary
  UHF Energy:          -74.05739918919758 Hartree
  Standard VQE Energy:   -74.08172177081973 Hartree
  FCI (Exact) Energy:  -74.0817233596675 Hartree
--------------------------------------------------
  Error (VQE vs FCI):    1.5888477804537615e-06 Hartree


In [4]:
import numpy as np
import time
from datetime import datetime

# --- Classical Chemistry Imports ---
from pyscf import gto, scf, fci

# --- Qiskit Nature Imports (Modern API) ---
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

# --- Qiskit Algorithm Imports (Modern API) ---
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import L_BFGS_B
from qiskit.primitives import Estimator

# ----- Configuration -----
BASIS_SET = "sto-3g"
BOND_LENGTH = 0.917  # Angstrom, experimental equilibrium bond length for HF

# ----- Main Calculation Function -----
def main():
    """
    Calculates the ground state energy of HF using three different methods:
    UHF, FCI, and standard VQE.
    """
    print(f"HF Ground State Calculation Comparison for Basis: {BASIS_SET}")
    print(f"Started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")

    # Define the molecule's geometry as a string
    atom_str = f"H 0 0 0; F 0 0 {BOND_LENGTH}"

    # Dictionary to store results
    results = {}

    # --- 1. Classical Calculations (UHF and FCI) ---
    print("--- Running Classical Calculations ---")
    start_time = time.time()
    # Note: charge=0 and spin=0 for the neutral HF molecule
    mol = gto.M(atom=atom_str, basis=BASIS_SET, charge=0, spin=0, verbose=0)
    
    # UHF Calculation
    mf_uhf = scf.UHF(mol).run()
    results['UHF'] = mf_uhf.e_tot
    # MODIFIED: Removed rounding
    print(f"UHF Energy: {results['UHF']} Hartree")
    
    # FCI Calculation
    mf_rhf = scf.RHF(mol).run()
    results['FCI'] = fci.FCI(mol, mf_rhf.mo_coeff).kernel()[0]
    # MODIFIED: Removed rounding
    print(f"FCI (Exact) Energy: {results['FCI']} Hartree")
    print(f"Classical calculations finished in {time.time() - start_time:.2f}s\n")

    # --- 2. Quantum Calculation (VQE) ---
    
    # Set up the chemical problem for Qiskit, specifying charge=0
    driver = PySCFDriver(atom=atom_str, basis=BASIS_SET, charge=0, spin=0)
    problem = driver.run()
    mapper = JordanWignerMapper()

    # Define the UCCSD ansatz
    ansatz = UCCSD(
        problem.num_spatial_orbitals,
        problem.num_particles,
        mapper,
        initial_state=HartreeFock(
            problem.num_spatial_orbitals,
            problem.num_particles,
            mapper,
        ),
    )

    # Define the VQE instance
    vqe_instance = VQE(Estimator(), ansatz, L_BFGS_B())
    vqe_instance.initial_point = np.zeros(ansatz.num_parameters)

    # --- Standard VQE Calculation ---
    print("--- Running Standard VQE Calculation ---")
    start_time = time.time()
    solver_vqe = GroundStateEigensolver(mapper, vqe_instance)
    result_vqe = solver_vqe.solve(problem)
    results['VQE'] = result_vqe.total_energies[0]
    # MODIFIED: Removed rounding
    print(f"Standard VQE Energy: {results['VQE']} Hartree")
    print(f"VQE finished in {time.time() - start_time:.2f}s\n")
    
    # --- Final Summary ---
    print("\n" + "="*50)
    print("               Final Energy Summary")
    print("="*50)
    # MODIFIED: Removed all rounding
    print(f"  UHF Energy:          {results['UHF']} Hartree")
    print(f"  Standard VQE Energy:   {results['VQE']} Hartree")
    print(f"  FCI (Exact) Energy:  {results['FCI']} Hartree")
    print("-" * 50)
    # Compare quantum results to the exact FCI energy
    print(f"  Error (VQE vs FCI):    {abs(results['VQE'] - results['FCI'])} Hartree")
    print("="*50)

if __name__ == "__main__":
    main()

HF Ground State Calculation Comparison for Basis: sto-3g
Started at: 2025-09-16 16:02:07

--- Running Classical Calculations ---
UHF Energy: -98.57077998601326 Hartree
FCI (Exact) Energy: -98.59662417998753 Hartree
Classical calculations finished in 0.61s



/tmp/ipykernel_399149/640875300.py:77: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  vqe_instance = VQE(Estimator(), ansatz, L_BFGS_B())


--- Running Standard VQE Calculation ---
Standard VQE Energy: -98.59662417984445 Hartree
VQE finished in 1503.31s


               Final Energy Summary
  UHF Energy:          -98.57077998601326 Hartree
  Standard VQE Energy:   -98.59662417984445 Hartree
  FCI (Exact) Energy:  -98.59662417998753 Hartree
--------------------------------------------------
  Error (VQE vs FCI):    1.4307488527265377e-10 Hartree


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
from datetime import datetime

# --- Qiskit Imports ---
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit_nature.second_q.transformers import FreezeCoreTransformer, ActiveSpaceTransformer

# For the VQE algorithm
from qiskit.primitives import Estimator
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import L_BFGS_B

# --- PySCF Imports for Classical Calculation ---
from pyscf import gto, scf

# ----- Helper Functions -----

def get_pyscf_mol(distance, basis, charge, spin):
    """
    Creates a PySCF molecule object for O₂.
    """
    mol = gto.Mole()
    # MOLECULE: O₂ centered at the origin
    mol.atom = f"O 0 0 {-distance/2}; O 0 0 {distance/2}"
    mol.basis = basis
    mol.charge = charge
    mol.spin = spin # Spin=2 for O₂ triplet ground state
    mol.build()
    return mol

def get_qiskit_problem(distance, basis, charge, spin):
    """
    Creates a Qiskit Nature ElectronicStructureProblem for O₂.
    """
    driver = PySCFDriver(
        atom=f"O 0 0 {-distance/2}; O 0 0 {distance/2}",
        unit=DistanceUnit.ANGSTROM,
        charge=charge,
        spin=spin,
        basis=basis
    )
    return driver.run()

def run_uhf_pyscf(mol):
    """Runs a classical UHF calculation."""
    mf = scf.UHF(mol).run(verbose=0)
    return mf.e_tot

def run_vqe_cpu(problem, mapper):
    """
    Runs the VQE algorithm on a local CPU-based simulator.
    """
    estimator = Estimator()
    optimizer = L_BFGS_B(maxiter=1000)
    initial_state = HartreeFock(problem.num_spatial_orbitals, problem.num_particles, mapper)
    ansatz = UCCSD(problem.num_spatial_orbitals, problem.num_particles, mapper, initial_state=initial_state)
    
    vqe = VQE(estimator, ansatz, optimizer)
    
    qubit_op = mapper.map(problem.hamiltonian.second_q_op())
    
    result = vqe.compute_minimum_eigenvalue(qubit_op)
    energy = problem.interpret(result).total_energies[0].real
    return energy

# --- Main Calculation ---

def main():
    """Main execution function."""
    print(f"🔬 O₂ Ground State Calculation with Active Space")
    print(f"Started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    
    # ----- Configuration -----
    # Using a minimal basis set for faster execution
    basis = 'sto-3g' 
    charge = 0
    spin = 2  # O₂ has a triplet ground state (2 unpaired electrons)
    dists = np.linspace(0.9, 3.0, 50)
    
    # Active Space Configuration for O₂ (a common choice)
    active_electrons = 8
    active_orbitals = 6

    # Lists to store results
    vqe_energies, uhf_energies = [], []

    # Loop through each bond distance
    for i, d in enumerate(dists):
        print(f"Calculating O₂ at d = {d:.2f} Å ({i+1}/{len(dists)})")
        
        # Classical UHF calculation
        mol = get_pyscf_mol(d, basis, charge, spin)
        uhf_energy = run_uhf_pyscf(mol)
        
        # NOTE: FCI calculation is omitted as it is computationally intractable.
        
        # Set up the quantum problem
        problem = get_qiskit_problem(d, basis, charge, spin)
        
        # --- APPLY TRANSFORMATIONS ---
        # 1. Freeze the core 1s orbitals of both Oxygen atoms
        freeze_transformer = FreezeCoreTransformer()
        problem = freeze_transformer.transform(problem)

        # 2. Select a chemically-active space from the valence orbitals
        active_transformer = ActiveSpaceTransformer(
            num_electrons=active_electrons,
            num_spatial_orbitals=active_orbitals
        )
        problem = active_transformer.transform(problem)
        # -----------------------------
        
        mapper = JordanWignerMapper()
        vqe_energy = run_vqe_cpu(problem, mapper)
        
        vqe_energies.append(vqe_energy)
        uhf_energies.append(uhf_energy)
        print(f"  VQE={vqe_energy:.6f} | UHF={uhf_energy:.6f}")

    print(f"\n--- Calculation Complete ---")

    # --- Plotting the Results (Using your requested style) ---
    plt.figure(figsize=(9, 5))

    plt.plot(dists, vqe_energies, '-', label='VQE', color='green', linewidth=1)
    plt.plot(dists, uhf_energies, '-', label='UHF', color='red', linewidth=1)
    # The FCI line is removed from the plot
    
    plt.xlabel("O-O Distance (Å)", fontsize=12)
    plt.ylabel("Energy (Hartree)", fontsize=12)
    plt.title("O₂ Potential Energy Surface", fontsize=14)
    plt.legend(fontsize=11)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    main()

In [ ]:
import numpy as np
import time
from datetime import datetime

# --- Classical Chemistry Imports ---
from pyscf import gto, scf, fci

# --- Qiskit Nature Imports (Modern API) ---
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

# --- Qiskit Algorithm Imports (Modern API) ---
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import L_BFGS_B
from qiskit.primitives import Estimator

# ----- Configuration -----
BASIS_SET = "sto-3g"  # Minimal basis set for demonstration
# Standard H2O geometry (in Angstroms)
# O is at origin, H atoms are positioned with typical bond length and angle
O_H_BOND_LENGTH = 0.96  # Angstrom
H_O_H_ANGLE = 104.5     # Degrees

# ----- Main Calculation Function -----
def main():
    """
    Calculates the ground state energy of H2O using three different methods:
    UHF, FCI, and standard VQE.
    """
    print(f"H₂O Ground State Calculation for Basis: {BASIS_SET}")
    print(f"Started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")

    # Calculate H positions based on bond length and angle
    # Convert angle to radians for calculations
    angle_rad = H_O_H_ANGLE * np.pi / 180.0
    
    # Calculate coordinates for hydrogen atoms
    h1_x = O_H_BOND_LENGTH * np.sin(angle_rad/2)
    h1_z = O_H_BOND_LENGTH * np.cos(angle_rad/2)
    h2_x = -h1_x  # Second H is mirrored across z-axis
    h2_z = h1_z

    # Define the molecule's geometry
    atom_str = f"O 0 0 0; H {h1_x} 0 {h1_z}; H {h2_x} 0 {h2_z}"
    print(f"Molecular geometry:\n{atom_str}\n")

    # Dictionary to store results
    results = {}

    # --- 1. Classical Calculations (UHF and FCI) ---
    print("--- Running Classical Calculations ---")
    start_time = time.time()
    mol = gto.M(atom=atom_str, basis=BASIS_SET, verbose=0)
    
    # UHF Calculation
    mf_uhf = scf.UHF(mol).run()
    results['UHF'] = mf_uhf.e_tot
    print(f"UHF Energy: {results['UHF']:.8f} Hartree")
    
    # FCI Calculation 
    # For H2O, FCI becomes more computationally demanding than for H2
    # We'll use RHF (Restricted Hartree-Fock) as a starting point for FCI
    mf_rhf = scf.RHF(mol).run()
    cisolver = fci.FCI(mol, mf_rhf.mo_coeff)
    
    print("Starting FCI calculation (this may take longer)...")
    results['FCI'] = cisolver.kernel()[0]
    print(f"FCI (Exact) Energy: {results['FCI']:.8f} Hartree")
    print(f"Classical calculations finished in {time.time() - start_time:.2f}s\n")

    # --- 2. Quantum Calculations (VQE) ---
    print("--- Running VQE Calculation ---")
    start_time = time.time()
    
    # Set up the chemical problem for Qiskit
    driver = PySCFDriver(atom=atom_str, basis=BASIS_SET)
    problem = driver.run()
    mapper = JordanWignerMapper()

    # Define the UCCSD ansatz for VQE
    ansatz = UCCSD(
        problem.num_spatial_orbitals,
        problem.num_particles,
        mapper,
        initial_state=HartreeFock(
            problem.num_spatial_orbitals,
            problem.num_particles,
            mapper,
        ),
    )
    
    # Using L-BFGS-B optimizer with a reasonable iteration limit
    optimizer = L_BFGS_B(maxiter=200)
    
    # Define the VQE instance
    vqe_instance = VQE(Estimator(), ansatz, optimizer)
    vqe_instance.initial_point = np.zeros(ansatz.num_parameters)

    # Run standard VQE
    solver_vqe = GroundStateEigensolver(mapper, vqe_instance)
    result_vqe = solver_vqe.solve(problem)
    results['VQE'] = result_vqe.total_energies[0]
    print(f"Standard VQE Energy: {results['VQE']:.8f} Hartree")
    print(f"VQE finished in {time.time() - start_time:.2f}s\n")

    # --- Final Summary ---
    print("\n" + "="*50)
    print("          Final Energy Summary for H₂O")
    print("="*50)
    print(f"  UHF Energy:           {results['UHF']:.8f} Hartree")
    print(f"  VQE Energy:           {results['VQE']:.8f} Hartree")
    print(f"  FCI (Exact) Energy:   {results['FCI']:.8f} Hartree")
    print("-" * 50)
    # Compare quantum results to the exact FCI energy
    print(f"  Error (VQE vs FCI):   {abs(results['VQE'] - results['FCI']):.8f} Hartree")
    print("="*50)

if __name__ == "__main__":
    main()

H₂O Ground State Calculation for Basis: sto-3g
Started at: 2025-09-19 10:20:57

Molecular geometry:
O 0 0 0; H 0.7590619907940895 0 0.5877285888330713; H -0.7590619907940895 0 0.5877285888330713

--- Running Classical Calculations ---
UHF Energy: -74.96331905 Hartree
Starting FCI calculation (this may take longer)...
FCI (Exact) Energy: -75.01315470 Hartree
Classical calculations finished in 1.49s

--- Running VQE Calculation ---


/tmp/ipykernel_1613161/2026379297.py:98: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  vqe_instance = VQE(Estimator(), ansatz, optimizer)


In [1]:
import numpy as np
import time
from datetime import datetime

# --- Classical Chemistry Imports ---
from pyscf import gto, scf, fci

# --- Qiskit Nature Imports (Modern API) ---
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
# NEW: Import the transformers
from qiskit_nature.second_q.transformers import FreezeCoreTransformer, ActiveSpaceTransformer

# --- Qiskit Algorithm Imports (Modern API) ---
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import L_BFGS_B
from qiskit.primitives import Estimator

# ----- Configuration -----
BASIS_SET = "sto-3g"

# ----- Main Calculation Function -----
def main():
    print(f"H₂O Ground State Calculation for Basis: {BASIS_SET}")
    print("Using FreezeCore and ActiveSpace Transformations\n")

    atom_str = "O 0.0 0.0 0.0; H 0.757 0.586 0.0; H -0.757 0.586 0.0"
    results = {}

    # --- 1. Classical Calculations (Full System for Reference) ---
    print("--- Running Full System Classical Calculations (for comparison) ---")
    start_time = time.time()
    mol = gto.M(atom=atom_str, basis=BASIS_SET, verbose=0)
    mf_rhf = scf.RHF(mol).run()
    results['FCI_Full'] = fci.FCI(mol, mf_rhf.mo_coeff).kernel()[0]
    print(f"Full System FCI (Exact) Energy: {results['FCI_Full']:.8f} Hartree")
    print(f"Classical calculations finished in {time.time() - start_time:.2f}s\n")

    # --- 2. Defining and Transforming the Quantum Problem ---
    print("--- Defining and Transforming the Chemical Problem ---")
    driver = PySCFDriver(atom=atom_str, basis=BASIS_SET)
    problem_full = driver.run()

    # Step A: Apply the FreezeCoreTransformer
    # This removes core orbitals and electrons (e.g., Oxygen 1s)
    transformer_freeze_core = FreezeCoreTransformer(freeze_core=True)
    problem_core_frozen = transformer_freeze_core.transform(problem_full)

    # Step B: Apply the ActiveSpaceTransformer to the already frozen problem
    # We select an active space of 4 electrons in 4 spatial orbitals
    transformer_active_space = ActiveSpaceTransformer(
        num_electrons=4,
        num_spatial_orbitals=4
    )
    problem_final = transformer_active_space.transform(problem_core_frozen)

    print(f"Original problem: {problem_full.num_particles} electrons in {problem_full.num_spatial_orbitals} orbitals.")
    print(f"After FreezeCore: {problem_core_frozen.num_particles} electrons in {problem_core_frozen.num_spatial_orbitals} orbitals.")
    print(f"After ActiveSpace: {problem_final.num_particles} electrons in {problem_final.num_spatial_orbitals} active orbitals.")
    print("-" * 50)

    # --- 3. Quantum Calculation (VQE on the Reduced Problem) ---
    print("\n--- Running VQE on the Reduced (Active Space) Problem ---")
    start_time = time.time()
    
    mapper = JordanWignerMapper()
    
    ansatz = UCCSD(
        problem_final.num_spatial_orbitals,
        problem_final.num_particles,
        mapper,
        initial_state=HartreeFock(
            problem_final.num_spatial_orbitals,
            problem_final.num_particles,
            mapper,
        ),
    )
    
    optimizer = L_BFGS_B(maxiter=1000)
    vqe_instance = VQE(Estimator(), ansatz, optimizer)
    vqe_instance.initial_point = np.zeros(ansatz.num_parameters)

    # The solver now runs on the final, transformed problem
    solver_vqe = GroundStateEigensolver(mapper, vqe_instance)
    result_vqe = solver_vqe.solve(problem_final)
    results['VQE_Transformed'] = result_vqe.total_energies[0]
    
    hamiltonian_op = problem_final.second_q_ops()[0]
    qubit_op = mapper.map(hamiltonian_op)
    print(f"Number of qubits required for transformed problem: {qubit_op.num_qubits}")
    print(f"VQE finished in {time.time() - start_time:.2f}s\n")

    # --- Final Summary ---
    print("\n" + "="*50)
    print("           Final Energy Summary for H₂O")
    print("="*50)
    print(f"  VQE (Transformed) Energy: {results['VQE_Transformed']:.8f} Hartree")
    print(f"  FCI (Full System) Energy: {results['FCI_Full']:.8f} Hartree")
    print("-" * 50)
    # The error is due to the active space approximation, not the VQE algorithm itself
    error = abs(results['VQE_Transformed'] - results['FCI_Full'])
    print(f"  Error (VQE vs Full FCI):  {error:.8f} Hartree")
    print("="*50)

if __name__ == "__main__":
    main()

H₂O Ground State Calculation for Basis: sto-3g
Using FreezeCore and ActiveSpace Transformations

--- Running Full System Classical Calculations (for comparison) ---
Full System FCI (Exact) Energy: -75.01243743 Hartree
Classical calculations finished in 1.49s

--- Defining and Transforming the Chemical Problem ---
Original problem: (5, 5) electrons in 7 orbitals.
After FreezeCore: (4, 4) electrons in 6 orbitals.
After ActiveSpace: (2, 2) electrons in 4 active orbitals.
--------------------------------------------------

--- Running VQE on the Reduced (Active Space) Problem ---


/tmp/ipykernel_1626325/4220385253.py:82: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  vqe_instance = VQE(Estimator(), ansatz, optimizer)


Number of qubits required for transformed problem: 8
VQE finished in 62.59s


           Final Energy Summary for H₂O
  VQE (Transformed) Energy: -74.97036252 Hartree
  FCI (Full System) Energy: -75.01243743 Hartree
--------------------------------------------------
  Error (VQE vs Full FCI):  0.04207491 Hartree


In [4]:
import numpy as np
import time
from datetime import datetime

# --- Classical Chemistry Imports ---
from pyscf import gto, scf, fci

# --- Qiskit Nature Imports (Modern API) ---
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit_nature.second_q.transformers import FreezeCoreTransformer, ActiveSpaceTransformer

# --- Qiskit Algorithm Imports (Old API - as in your working code) ---
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import L_BFGS_B
from qiskit.primitives import Estimator

# ----- Configuration -----
BASIS_SET = "sto-3g"

# ----- Main Calculation Function -----
def main():
    print(f"H₂O Ground State Calculation for Basis: {BASIS_SET}")
    print("Using ONLY FreezeCore Transformation (all valence electrons active)\n")

    atom_str = "O 0.0 0.0 0.0; H 0.757 0.586 0.0; H -0.757 0.586 0.0"
    results = {}

    # --- 1. Classical Calculations (Full System for Reference) ---
    print("--- Running Full System Classical Calculations (for comparison) ---")
    start_time = time.time()
    mol = gto.M(atom=atom_str, basis=BASIS_SET, verbose=0)
    mf_rhf = scf.RHF(mol).run()
    results['FCI_Full'] = fci.FCI(mol, mf_rhf.mo_coeff).kernel()[0]
    print(f"Full System FCI (Exact) Energy: {results['FCI_Full']:.8f} Hartree")
    print(f"Classical calculations finished in {time.time() - start_time:.2f}s\n")

    # --- 2. Defining and Transforming the Quantum Problem ---
    print("--- Defining and Transforming the Chemical Problem ---")
    driver = PySCFDriver(atom=atom_str, basis=BASIS_SET)
    problem_full = driver.run()

    # Step A: Apply the FreezeCoreTransformer
    # This removes core orbitals and electrons (e.g., Oxygen 1s)
    transformer_freeze_core = FreezeCoreTransformer(freeze_core=True)
    problem_core_frozen = transformer_freeze_core.transform(problem_full)

    # CHANGED: The ActiveSpaceTransformer has been removed.
    # The final problem now includes all valence electrons.
    problem_final = problem_core_frozen

    print(f"Original problem: {problem_full.num_particles} electrons in {problem_full.num_spatial_orbitals} orbitals.")
    print(f"Final problem after FreezeCore: {problem_final.num_particles} electrons in {problem_final.num_spatial_orbitals} orbitals.")
    print("-" * 50)

    # --- 3. Quantum Calculation (VQE on the Reduced Problem) ---
    print("\n--- Running VQE on the Reduced (All Valence) Problem ---")
    start_time = time.time()
    
    mapper = JordanWignerMapper()
    
    ansatz = UCCSD(
        problem_final.num_spatial_orbitals,
        problem_final.num_particles,
        mapper,
        initial_state=HartreeFock(
            problem_final.num_spatial_orbitals,
            problem_final.num_particles,
            mapper,
        ),
    )
    
    optimizer = L_BFGS_B(maxiter=1000)
    vqe_instance = VQE(Estimator(), ansatz, optimizer)
    vqe_instance.initial_point = np.zeros(ansatz.num_parameters)

    # The solver now runs on the final, transformed problem
    solver_vqe = GroundStateEigensolver(mapper, vqe_instance)
    result_vqe = solver_vqe.solve(problem_final)
    results['VQE_Transformed'] = result_vqe.total_energies[0]
    
    hamiltonian_op = problem_final.second_q_ops()[0]
    qubit_op = mapper.map(hamiltonian_op)
    print(f"Number of qubits required for transformed problem: {qubit_op.num_qubits}")
    print(f"VQE finished in {time.time() - start_time:.2f}s\n")

    # --- Final Summary ---
    print("\n" + "="*50)
    print("           Final Energy Summary for H₂O")
    print("="*50)
    print(f"  VQE (Transformed) Energy: {results['VQE_Transformed']:.8f} Hartree")
    print(f"  FCI (Full System) Energy: {results['FCI_Full']:.8f} Hartree")
    print("-" * 50)
    error = abs(results['VQE_Transformed'] - results['FCI_Full'])
    print(f"  Error (VQE vs Full FCI):  {error:.8f} Hartree")
    print("="*50)

if __name__ == "__main__":
    main()

H₂O Ground State Calculation for Basis: sto-3g
Using ONLY FreezeCore Transformation (all valence electrons active)

--- Running Full System Classical Calculations (for comparison) ---
Full System FCI (Exact) Energy: -75.01243743 Hartree
Classical calculations finished in 0.50s

--- Defining and Transforming the Chemical Problem ---
Original problem: (5, 5) electrons in 7 orbitals.
Final problem after FreezeCore: (4, 4) electrons in 6 orbitals.
--------------------------------------------------

--- Running VQE on the Reduced (All Valence) Problem ---


/tmp/ipykernel_1626325/332603793.py:76: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  vqe_instance = VQE(Estimator(), ansatz, optimizer)


Number of qubits required for transformed problem: 12
VQE finished in 3342.46s


           Final Energy Summary for H₂O
  VQE (Transformed) Energy: -75.01225875 Hartree
  FCI (Full System) Energy: -75.01243743 Hartree
--------------------------------------------------
  Error (VQE vs Full FCI):  0.00017868 Hartree


In [6]:
import numpy as np
import time
from datetime import datetime

# --- Classical Chemistry Imports ---
from pyscf import gto, scf, fci

# --- Qiskit Nature Imports (Modern API) ---
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit_nature.second_q.transformers import FreezeCoreTransformer, ActiveSpaceTransformer

# --- Qiskit Algorithm Imports ---
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import L_BFGS_B
from qiskit.primitives import Estimator

# ----- Configuration -----
BASIS_SET = "sto-3g"

# ----- Main Calculation Function -----
def main():
    """
    Calculates the ground state energy of CH4 using FCI and VQE with a
    FreezeCore and a (4e, 4o) ActiveSpace transformation.
    """
    print(f"CH₄ Ground State Calculation for Basis: {BASIS_SET}")
    print("Using FreezeCore and a (4, 4) Active Space Transformation\n")

    # CHANGED: Corrected the formatting of the atom string
    atom_str = (
        "C 0.000000 0.000000 0.000000; "
        "H 0.627638 0.627638 0.627638; "
        "H 0.627638 -0.627638 -0.627638; "
        "H -0.627638 0.627638 -0.627638; "
        "H -0.627638 -0.627638 0.627638"
    )
    results = {}

    # --- 1. Classical Calculations (Full System for Reference) ---
    print("--- Running Full System Classical Calculations (for comparison) ---")
    start_time = time.time()
    mol = gto.M(atom=atom_str, basis=BASIS_SET, verbose=0)
    mf_rhf = scf.RHF(mol).run()
    results['FCI_Full'] = fci.FCI(mol, mf_rhf.mo_coeff).kernel()[0]
    print(f"Full System FCI (Exact) Energy: {results['FCI_Full']:.8f} Hartree")
    print(f"Classical calculations finished in {time.time() - start_time:.2f}s\n")

    # --- 2. Defining and Transforming the Quantum Problem ---
    print("--- Defining and Transforming the Chemical Problem ---")
    driver = PySCFDriver(atom=atom_str, basis=BASIS_SET)
    problem_full = driver.run()

    # Step A: Apply the FreezeCoreTransformer
    transformer_freeze_core = FreezeCoreTransformer(freeze_core=True)
    problem_core_frozen = transformer_freeze_core.transform(problem_full)

    # Step B: Apply the ActiveSpaceTransformer to the already frozen problem
    transformer_active_space = ActiveSpaceTransformer(
        num_electrons=4,
        num_spatial_orbitals=4
    )
    problem_final = transformer_active_space.transform(problem_core_frozen)

    print(f"Original problem: {problem_full.num_particles} electrons in {problem_full.num_spatial_orbitals} orbitals.")
    print(f"After FreezeCore: {problem_core_frozen.num_particles} electrons in {problem_core_frozen.num_spatial_orbitals} orbitals.")
    print(f"After ActiveSpace (4,4): {problem_final.num_particles} electrons in {problem_final.num_spatial_orbitals} active orbitals.")
    print("-" * 50)

    # --- 3. Quantum Calculation (VQE on the Reduced Problem) ---
    print("\n--- Running VQE on the Reduced (4, 4) Active Space ---")
    start_time = time.time()
    
    mapper = JordanWignerMapper()
    
    ansatz = UCCSD(
        problem_final.num_spatial_orbitals,
        problem_final.num_particles,
        mapper,
        initial_state=HartreeFock(
            problem_final.num_spatial_orbitals,
            problem_final.num_particles,
            mapper,
        ),
    )
    
    optimizer = L_BFGS_B(maxiter=1000)
    vqe_instance = VQE(Estimator(), ansatz, optimizer)
    vqe_instance.initial_point = np.zeros(ansatz.num_parameters)

    # The solver now runs on the final, transformed problem
    solver_vqe = GroundStateEigensolver(mapper, vqe_instance)
    result_vqe = solver_vqe.solve(problem_final)
    results['VQE_Transformed'] = result_vqe.total_energies[0]
    
    hamiltonian_op = problem_final.second_q_ops()[0]
    qubit_op = mapper.map(hamiltonian_op)
    print(f"Number of qubits required for transformed problem: {qubit_op.num_qubits}")
    print(f"VQE finished in {time.time() - start_time:.2f}s\n")

    # --- Final Summary ---
    print("\n" + "="*50)
    print("           Final Energy Summary for CH₄")
    print("="*50)
    print(f"  VQE (Transformed) Energy: {results['VQE_Transformed']:.8f} Hartree")
    print(f"  FCI (Full System) Energy: {results['FCI_Full']:.8f} Hartree")
    print("-" * 50)
    error = abs(results['VQE_Transformed'] - results['FCI_Full'])
    print(f"  Error (VQE vs Full FCI):  {error:.8f} Hartree")
    print("="*50)

if __name__ == "__main__":
    main()

CH₄ Ground State Calculation for Basis: sto-3g
Using FreezeCore and a (4, 4) Active Space Transformation

--- Running Full System Classical Calculations (for comparison) ---
Full System FCI (Exact) Energy: -39.80569367 Hartree
Classical calculations finished in 0.57s

--- Defining and Transforming the Chemical Problem ---
Original problem: (5, 5) electrons in 9 orbitals.
After FreezeCore: (4, 4) electrons in 8 orbitals.
After ActiveSpace (4,4): (2, 2) electrons in 4 active orbitals.
--------------------------------------------------

--- Running VQE on the Reduced (4, 4) Active Space ---


/tmp/ipykernel_1626325/4173579592.py:90: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  vqe_instance = VQE(Estimator(), ansatz, optimizer)


Number of qubits required for transformed problem: 8
VQE finished in 65.59s


           Final Energy Summary for CH₄
  VQE (Transformed) Energy: -39.73753061 Hartree
  FCI (Full System) Energy: -39.80569367 Hartree
--------------------------------------------------
  Error (VQE vs Full FCI):  0.06816306 Hartree


In [7]:
import numpy as np
import time
from datetime import datetime

# --- Classical Chemistry Imports ---
from pyscf import gto, scf, fci

# --- Qiskit Nature Imports (Modern API) ---
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit_nature.second_q.transformers import FreezeCoreTransformer, ActiveSpaceTransformer

# --- Qiskit Algorithm Imports ---
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import L_BFGS_B
from qiskit.primitives import Estimator

# ----- Configuration -----
BASIS_SET = "sto-3g"

# ----- Main Calculation Function -----
def main():
    """
    Calculates the ground state energy of NH3 using FCI and VQE with a
    FreezeCore and a (4e, 4o) ActiveSpace transformation.
    """
    print(f"NH₃ Ground State Calculation for Basis: {BASIS_SET}")
    print("Using FreezeCore and a (4, 4) Active Space Transformation\n")

    # Define the ammonia molecule's geometry (trigonal pyramidal) in Angstroms
    atom_str = (
        "N 0.000000 0.000000 0.116219; "
        "H 0.000000 0.938561 -0.271178; "
        "H 0.812814 -0.469281 -0.271178; "
        "H -0.812814 -0.469281 -0.271178"
    )
    results = {}

    # --- 1. Classical Calculations (Full System for Reference) ---
    print("--- Running Full System Classical Calculations (for comparison) ---")
    start_time = time.time()
    mol = gto.M(atom=atom_str, basis=BASIS_SET, verbose=0)
    mf_rhf = scf.RHF(mol).run()
    results['FCI_Full'] = fci.FCI(mol, mf_rhf.mo_coeff).kernel()[0]
    print(f"Full System FCI (Exact) Energy: {results['FCI_Full']:.8f} Hartree")
    print(f"Classical calculations finished in {time.time() - start_time:.2f}s\n")

    # --- 2. Defining and Transforming the Quantum Problem ---
    print("--- Defining and Transforming the Chemical Problem ---")
    driver = PySCFDriver(atom=atom_str, basis=BASIS_SET)
    problem_full = driver.run()

    # Step A: Apply the FreezeCoreTransformer
    transformer_freeze_core = FreezeCoreTransformer(freeze_core=True)
    problem_core_frozen = transformer_freeze_core.transform(problem_full)

    # Step B: Apply the ActiveSpaceTransformer to the already frozen problem
    transformer_active_space = ActiveSpaceTransformer(
        num_electrons=4,
        num_spatial_orbitals=4
    )
    problem_final = transformer_active_space.transform(problem_core_frozen)

    print(f"Original problem: {problem_full.num_particles} electrons in {problem_full.num_spatial_orbitals} orbitals.")
    print(f"After FreezeCore: {problem_core_frozen.num_particles} electrons in {problem_core_frozen.num_spatial_orbitals} orbitals.")
    print(f"After ActiveSpace (4,4): {problem_final.num_particles} electrons in {problem_final.num_spatial_orbitals} active orbitals.")
    print("-" * 50)

    # --- 3. Quantum Calculation (VQE on the Reduced Problem) ---
    print("\n--- Running VQE on the Reduced (4, 4) Active Space ---")
    start_time = time.time()
    
    mapper = JordanWignerMapper()
    
    ansatz = UCCSD(
        problem_final.num_spatial_orbitals,
        problem_final.num_particles,
        mapper,
        initial_state=HartreeFock(
            problem_final.num_spatial_orbitals,
            problem_final.num_particles,
            mapper,
        ),
    )
    
    optimizer = L_BFGS_B(maxiter=1000)
    vqe_instance = VQE(Estimator(), ansatz, optimizer)
    vqe_instance.initial_point = np.zeros(ansatz.num_parameters)

    # The solver now runs on the final, transformed problem
    solver_vqe = GroundStateEigensolver(mapper, vqe_instance)
    result_vqe = solver_vqe.solve(problem_final)
    results['VQE_Transformed'] = result_vqe.total_energies[0]
    
    hamiltonian_op = problem_final.second_q_ops()[0]
    qubit_op = mapper.map(hamiltonian_op)
    print(f"Number of qubits required for transformed problem: {qubit_op.num_qubits}")
    print(f"VQE finished in {time.time() - start_time:.2f}s\n")

    # --- Final Summary ---
    print("\n" + "="*50)
    print("           Final Energy Summary for NH₃")
    print("="*50)
    print(f"  VQE (Transformed) Energy: {results['VQE_Transformed']:.8f} Hartree")
    print(f"  FCI (Full System) Energy: {results['FCI_Full']:.8f} Hartree")
    print("-" * 50)
    error = abs(results['VQE_Transformed'] - results['FCI_Full'])
    print(f"  Error (VQE vs Full FCI):  {error:.8f} Hartree")
    print("="*50)

if __name__ == "__main__":
    main()

NH₃ Ground State Calculation for Basis: sto-3g
Using FreezeCore and a (4, 4) Active Space Transformation

--- Running Full System Classical Calculations (for comparison) ---
Full System FCI (Exact) Energy: -55.52012266 Hartree
Classical calculations finished in 0.53s

--- Defining and Transforming the Chemical Problem ---
Original problem: (5, 5) electrons in 8 orbitals.
After FreezeCore: (4, 4) electrons in 7 orbitals.
After ActiveSpace (4,4): (2, 2) electrons in 4 active orbitals.
--------------------------------------------------

--- Running VQE on the Reduced (4, 4) Active Space ---


/tmp/ipykernel_1626325/3863084115.py:89: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  vqe_instance = VQE(Estimator(), ansatz, optimizer)


Number of qubits required for transformed problem: 8
VQE finished in 72.50s


           Final Energy Summary for NH₃
  VQE (Transformed) Energy: -55.45981170 Hartree
  FCI (Full System) Energy: -55.52012266 Hartree
--------------------------------------------------
  Error (VQE vs Full FCI):  0.06031096 Hartree


In [8]:
import numpy as np
import time
from datetime import datetime

# --- Classical Chemistry Imports ---
from pyscf import gto, scf, fci

# --- Qiskit Nature Imports (Modern API) ---
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit_nature.second_q.transformers import FreezeCoreTransformer, ActiveSpaceTransformer

# --- Qiskit Algorithm Imports ---
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import L_BFGS_B
from qiskit.primitives import Estimator

# ----- Configuration -----
BASIS_SET = "sto-3g"

# ----- Main Calculation Function -----
def main():
    """
    Calculates the ground state energy of BH3 using FCI and VQE with a
    FreezeCore and a (4e, 4o) ActiveSpace transformation.
    """
    print(f"BH₃ Ground State Calculation for Basis: {BASIS_SET}")
    print("Using FreezeCore and a (4, 4) Active Space Transformation\n")

    # Define the borane molecule's geometry (trigonal planar) in Angstroms
    atom_str = (
        "B 0.000000  0.000000 0.000000; "
        "H 1.190000  0.000000 0.000000; "
        "H -0.595000  1.030600 0.000000; "
        "H -0.595000 -1.030600 0.000000"
    )
    results = {}

    # --- 1. Classical Calculations (Full System for Reference) ---
    print("--- Running Full System Classical Calculations (for comparison) ---")
    start_time = time.time()
    mol = gto.M(atom=atom_str, basis=BASIS_SET, verbose=0)
    mf_rhf = scf.RHF(mol).run()
    results['FCI_Full'] = fci.FCI(mol, mf_rhf.mo_coeff).kernel()[0]
    print(f"Full System FCI (Exact) Energy: {results['FCI_Full']:.8f} Hartree")
    print(f"Classical calculations finished in {time.time() - start_time:.2f}s\n")

    # --- 2. Defining and Transforming the Quantum Problem ---
    print("--- Defining and Transforming the Chemical Problem ---")
    driver = PySCFDriver(atom=atom_str, basis=BASIS_SET)
    problem_full = driver.run()

    # Step A: Apply the FreezeCoreTransformer
    transformer_freeze_core = FreezeCoreTransformer(freeze_core=True)
    problem_core_frozen = transformer_freeze_core.transform(problem_full)

    # Step B: Apply the ActiveSpaceTransformer to the already frozen problem
    transformer_active_space = ActiveSpaceTransformer(
        num_electrons=4,
        num_spatial_orbitals=4
    )
    problem_final = transformer_active_space.transform(problem_core_frozen)

    print(f"Original problem: {problem_full.num_particles} electrons in {problem_full.num_spatial_orbitals} orbitals.")
    print(f"After FreezeCore: {problem_core_frozen.num_particles} electrons in {problem_core_frozen.num_spatial_orbitals} orbitals.")
    print(f"After ActiveSpace (4,4): {problem_final.num_particles} electrons in {problem_final.num_spatial_orbitals} active orbitals.")
    print("-" * 50)

    # --- 3. Quantum Calculation (VQE on the Reduced Problem) ---
    print("\n--- Running VQE on the Reduced (4, 4) Active Space ---")
    start_time = time.time()
    
    mapper = JordanWignerMapper()
    
    ansatz = UCCSD(
        problem_final.num_spatial_orbitals,
        problem_final.num_particles,
        mapper,
        initial_state=HartreeFock(
            problem_final.num_spatial_orbitals,
            problem_final.num_particles,
            mapper,
        ),
    )
    
    optimizer = L_BFGS_B(maxiter=1000)
    vqe_instance = VQE(Estimator(), ansatz, optimizer)
    vqe_instance.initial_point = np.zeros(ansatz.num_parameters)

    # The solver now runs on the final, transformed problem
    solver_vqe = GroundStateEigensolver(mapper, vqe_instance)
    result_vqe = solver_vqe.solve(problem_final)
    results['VQE_Transformed'] = result_vqe.total_energies[0]
    
    hamiltonian_op = problem_final.second_q_ops()[0]
    qubit_op = mapper.map(hamiltonian_op)
    print(f"Number of qubits required for transformed problem: {qubit_op.num_qubits}")
    print(f"VQE finished in {time.time() - start_time:.2f}s\n")

    # --- Final Summary ---
    print("\n" + "="*50)
    print("           Final Energy Summary for BH₃")
    print("="*50)
    print(f"  VQE (Transformed) Energy: {results['VQE_Transformed']:.8f} Hartree")
    print(f"  FCI (Full System) Energy: {results['FCI_Full']:.8f} Hartree")
    print("-" * 50)
    error = abs(results['VQE_Transformed'] - results['FCI_Full'])
    print(f"  Error (VQE vs Full FCI):  {error:.8f} Hartree")
    print("="*50)

if __name__ == "__main__":
    main()

BH₃ Ground State Calculation for Basis: sto-3g
Using FreezeCore and a (4, 4) Active Space Transformation

--- Running Full System Classical Calculations (for comparison) ---
Full System FCI (Exact) Energy: -26.12237017 Hartree
Classical calculations finished in 0.38s

--- Defining and Transforming the Chemical Problem ---
Original problem: (4, 4) electrons in 8 orbitals.
After FreezeCore: (3, 3) electrons in 7 orbitals.
After ActiveSpace (4,4): (2, 2) electrons in 4 active orbitals.
--------------------------------------------------

--- Running VQE on the Reduced (4, 4) Active Space ---


/tmp/ipykernel_1626325/3720040170.py:89: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  vqe_instance = VQE(Estimator(), ansatz, optimizer)


Number of qubits required for transformed problem: 8
VQE finished in 55.58s


           Final Energy Summary for BH₃
  VQE (Transformed) Energy: -26.07375971 Hartree
  FCI (Full System) Energy: -26.12237017 Hartree
--------------------------------------------------
  Error (VQE vs Full FCI):  0.04861047 Hartree


In [ ]:
import numpy as np
import time
from datetime import datetime

# --- Classical Chemistry Imports ---
from pyscf import gto, scf, fci, ao2mo

# --- Qiskit Nature Imports (Modern API) ---
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
# NEW IMPORTS for manual construction
from qiskit_nature.second_q.hamiltonians import ElectronicEnergy
from qiskit_nature.second_q.problems import ElectronicStructureProblem

# --- Qiskit Algorithm Imports ---
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import L_BFGS_B
from qiskit.primitives import Estimator

# ----- Configuration -----
BASIS_SET = "sto-3g"

# ----- Main Calculation Function -----
def main():
    """
    Calculates the ground state energy of O2 by manually constructing the
    active space problem to bypass faulty transformers.
    """
    print(f"O₂ Ground State Calculation for Basis: {BASIS_SET}")
    print("Using MANUAL construction of the full valence active space\n")

    atom_str = "O 0.0 0.0 0.0; O 0.0 0.0 1.21"
    O2_SPIN = 2
    results = {}

    # --- 1. Classical Calculation (Full System) ---
    print("--- Running Full System Classical Calculation ---")
    start_time = time.time()
    mol = gto.M(atom=atom_str, basis=BASIS_SET, spin=O2_SPIN, verbose=0)
    mf_uhf = scf.UHF(mol).run()
    results['FCI_Full'] = fci.FCI(mol, mf_uhf.mo_coeff).kernel()[0]
    print(f"Full System FCI (Exact) Energy: {results['FCI_Full']:.8f} Hartree")
    print(f"Classical calculations finished in {time.time() - start_time:.2f}s\n")

    # --- 2. Manual Problem Construction (Bypassing Transformers) ---
    print("--- Manually Constructing the Active Space Problem ---")
    
    # Total number of orbitals and electrons from the PySCF calculation
    n_orbitals = mf_uhf.mo_coeff[0].shape[1]
    n_electrons_alpha = mol.nelec[0]
    n_electrons_beta = mol.nelec[1]

    # Define the active space manually
    # We freeze the 2 lowest energy orbitals (the 1s cores)
    # Our active space is the remaining 8 valence orbitals.
    n_core_orbitals = 2
    n_active_orbitals = n_orbitals - n_core_orbitals
    active_orbital_indices = list(range(n_core_orbitals, n_orbitals))
    
    # The number of electrons in the active space is also defined manually
    # We remove 2 alpha and 2 beta core electrons
    n_active_electrons_alpha = n_electrons_alpha - n_core_orbitals
    n_active_electrons_beta = n_electrons_beta - n_core_orbitals
    
    # Get the electronic integrals from PySCF
    h1_ao = mf_uhf.get_hcore()
    h2_ao = mf_uhf.get_veff() - mf_uhf.get_j() + mf_uhf.get_k() # In UHF, this is complex
    h2_ao = ao2mo.restore(1, mf_uhf._eri, n_orbitals) # Get full 2-electron integrals
    
    # Transform integrals into the MO basis for alpha and beta spins
    mo_a, mo_b = mf_uhf.mo_coeff
    h1_mo_a = mo_a.T @ h1_ao @ mo_a
    h1_mo_b = mo_b.T @ h1_ao @ mo_b
    h2_mo_aa = ao2mo.kernel(h2_ao, mo_a)
    h2_mo_bb = ao2mo.kernel(h2_ao, mo_b)
    h2_mo_ab = ao2mo.kernel(h2_ao, (mo_a, mo_a, mo_b, mo_b))

    # Select only the integrals corresponding to our active orbitals
    idx = np.ix_(active_orbital_indices, active_orbital_indices)
    h1_active_a = h1_mo_a[idx]
    h1_active_b = h1_mo_b[idx]
    idx_4 = np.ix_(active_orbital_indices, active_orbital_indices, 
                   active_orbital_indices, active_orbital_indices)
    h2_active_aa = h2_mo_aa[idx_4]
    h2_active_bb = h2_mo_bb[idx_4]
    h2_active_ab = h2_mo_ab[idx_4]

    # Manually create the electronic energy Hamiltonian
    electronic_hamiltonian = ElectronicEnergy.from_raw_integrals(
        h1_active_a, h2_active_aa, h1_active_b, h2_active_bb, h2_active_ab
    )
    electronic_hamiltonian.nuclear_repulsion_energy = mol.energy_nuc()
    
    # Manually create the final problem instance
    problem_final = ElectronicStructureProblem(electronic_hamiltonian)
    problem_final.num_spatial_orbitals = n_active_orbitals
    problem_final.num_particles = (n_active_electrons_alpha, n_active_electrons_beta)
    
    print(f"Final problem (manual): {problem_final.num_particles} electrons in {problem_final.num_spatial_orbitals} orbitals.")
    print("-" * 50)
    
    # --- 3. Quantum Calculation (VQE) ---
    print("\n--- Running VQE on the Manually Constructed Problem ---")
    print("NOTE: This 16-qubit calculation will be very slow.")
    start_time = time.time()
    
    mapper = JordanWignerMapper()
    ansatz = UCCSD(
        problem_final.num_spatial_orbitals,
        problem_final.num_particles,
        mapper,
        initial_state=HartreeFock(
            problem_final.num_spatial_orbitals,
            problem_final.num_particles,
            mapper,
        ),
    )
    
    optimizer = L_BFGS_B(maxiter=1000)
    vqe_instance = VQE(Estimator(), ansatz, optimizer)
    vqe_instance.initial_point = np.zeros(ansatz.num_parameters)

    solver_vqe = GroundStateEigensolver(mapper, vqe_instance)
    result_vqe = solver_vqe.solve(problem_final)
    results['VQE_Transformed'] = result_vqe.total_energies[0]
    
    hamiltonian_op = problem_final.second_q_ops()[0]
    qubit_op = mapper.map(hamiltonian_op)
    print(f"Number of qubits required: {qubit_op.num_qubits}")
    print(f"VQE finished in {time.time() - start_time:.2f}s\n")

    # --- Final Summary ---
    print("\n" + "="*50)
    print("           Final Energy Summary for O₂")
    print("="*50)
    print(f"  VQE (Manual Space) Energy: {results['VQE_Transformed']:.8f} Hartree")
    print(f"  FCI (Full System) Energy: {results['FCI_Full']:.8f} Hartree")
    print("-" * 50)
    error = abs(results['VQE_Transformed'] - results['FCI_Full'])
    print(f"  Error (VQE vs Full FCI):  {error:.8f} Hartree")
    print("="*50)

if __name__ == "__main__":
    main()

O₂ Ground State Calculation for Basis: sto-3g
Using MANUAL construction of the full valence active space

--- Running Full System Classical Calculation ---
Full System FCI (Exact) Energy: -147.74478939 Hartree
Classical calculations finished in 0.81s

--- Manually Constructing the Active Space Problem ---
Final problem (manual): (7, 5) electrons in 8 orbitals.
--------------------------------------------------

--- Running VQE on the Manually Constructed Problem ---
NOTE: This 16-qubit calculation will be very slow.


/tmp/ipykernel_1743814/3723315111.py:121: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  vqe_instance = VQE(Estimator(), ansatz, optimizer)


In [1]:
import numpy as np
import time
from datetime import datetime

# --- Classical Chemistry Imports ---
from pyscf import gto, scf, fci, ao2mo

# --- Qiskit Nature Imports (Modern API) ---
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit_nature.second_q.hamiltonians import ElectronicEnergy
from qiskit_nature.second_q.problems import ElectronicStructureProblem

# --- Qiskit Algorithm Imports ---
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import L_BFGS_B
from qiskit.primitives import Estimator

# ----- Configuration -----
BASIS_SET = "sto-3g"

# ----- Main Calculation Function -----
def main():
    """
    Calculates the ground state energy of O2 using a manually constructed
    p-orbital active space (8e, 6o) for a 12-qubit simulation.
    """
    print(f"O₂ Ground State Calculation for Basis: {BASIS_SET}")
    print("Using a manually constructed (8, 6) p-orbital active space\n")

    atom_str = "O 0.0 0.0 0.0; O 0.0 0.0 1.21"
    O2_SPIN = 2
    results = {}

    # --- 1. Classical Calculation (Full System) ---
    print("--- Running Full System Classical Calculation ---")
    start_time = time.time()
    mol = gto.M(atom=atom_str, basis=BASIS_SET, spin=O2_SPIN, verbose=0)
    mf_uhf = scf.UHF(mol).run()
    results['FCI_Full'] = fci.FCI(mol, mf_uhf.mo_coeff).kernel()[0]
    print(f"Full System FCI (Exact) Energy: {results['FCI_Full']:.8f} Hartree")
    print(f"Classical calculations finished in {time.time() - start_time:.2f}s\n")

    # --- 2. Manual Problem Construction (Bypassing Transformers) ---
    print("--- Manually Constructing the Active Space Problem ---")
    
    n_orbitals = mf_uhf.mo_coeff[0].shape[1]

    # CHANGED: Define a smaller active space for a 12-qubit problem
    # We now have 4 inactive orbitals (2 core + 2 low-lying valence)
    n_inactive_orbitals = 4
    n_active_orbitals = 6  # This will result in 12 qubits
    active_orbital_indices = list(range(n_inactive_orbitals, n_inactive_orbitals + n_active_orbitals))
    
    # CHANGED: Manually define the number of active electrons for this space
    # This corresponds to the (5 alpha, 3 beta) occupation of the p-shell
    n_active_electrons_alpha = 5
    n_active_electrons_beta = 3
    
    # Get and transform electronic integrals from PySCF
    h1_ao = mf_uhf.get_hcore()
    h2_ao = ao2mo.restore(1, mf_uhf._eri, n_orbitals)
    mo_a, mo_b = mf_uhf.mo_coeff
    h1_mo_a = mo_a.T @ h1_ao @ mo_a
    h1_mo_b = mo_b.T @ h1_ao @ mo_b
    h2_mo_aa = ao2mo.kernel(h2_ao, mo_a)
    h2_mo_bb = ao2mo.kernel(h2_ao, mo_b)
    h2_mo_ab = ao2mo.kernel(h2_ao, (mo_a, mo_a, mo_b, mo_b))

    # Select only the integrals corresponding to our active orbitals
    idx = np.ix_(active_orbital_indices, active_orbital_indices)
    h1_active_a = h1_mo_a[idx]
    h1_active_b = h1_mo_b[idx]
    idx_4 = np.ix_(active_orbital_indices, active_orbital_indices, 
                   active_orbital_indices, active_orbital_indices)
    h2_active_aa = h2_mo_aa[idx_4]
    h2_active_bb = h2_mo_bb[idx_4]
    h2_active_ab = h2_mo_ab[idx_4]

    # Manually create the electronic energy Hamiltonian
    electronic_hamiltonian = ElectronicEnergy.from_raw_integrals(
        h1_active_a, h2_active_aa, h1_active_b, h2_active_bb, h2_active_ab
    )
    electronic_hamiltonian.nuclear_repulsion_energy = mol.energy_nuc()
    
    # Manually create the final problem instance
    problem_final = ElectronicStructureProblem(electronic_hamiltonian)
    problem_final.num_spatial_orbitals = n_active_orbitals
    problem_final.num_particles = (n_active_electrons_alpha, n_active_electrons_beta)
    
    print(f"Final problem (manual): {problem_final.num_particles} electrons in {problem_final.num_spatial_orbitals} orbitals.")
    print("-" * 50)
    
    # --- 3. Quantum Calculation (VQE) ---
    print("\n--- Running VQE on the 12-Qubit Problem ---")
    start_time = time.time()
    
    mapper = JordanWignerMapper()
    ansatz = UCCSD(
        problem_final.num_spatial_orbitals,
        problem_final.num_particles,
        mapper,
        initial_state=HartreeFock(
            problem_final.num_spatial_orbitals,
            problem_final.num_particles,
            mapper,
        ),
    )
    
    optimizer = L_BFGS_B(maxiter=1000)
    vqe_instance = VQE(Estimator(), ansatz, optimizer)
    vqe_instance.initial_point = np.zeros(ansatz.num_parameters)

    solver_vqe = GroundStateEigensolver(mapper, vqe_instance)
    result_vqe = solver_vqe.solve(problem_final)
    results['VQE_Transformed'] = result_vqe.total_energies[0]
    
    hamiltonian_op = problem_final.second_q_ops()[0]
    qubit_op = mapper.map(hamiltonian_op)
    print(f"Number of qubits required: {qubit_op.num_qubits}")
    print(f"VQE finished in {time.time() - start_time:.2f}s\n")

    # --- Final Summary ---
    print("\n" + "="*50)
    print("           Final Energy Summary for O₂")
    print("="*50)
    print(f"  VQE (12-Qubit) Energy: {results['VQE_Transformed']:.8f} Hartree")
    print(f"  FCI (Full System) Energy: {results['FCI_Full']:.8f} Hartree")
    print("-" * 50)
    error = abs(results['VQE_Transformed'] - results['FCI_Full'])
    print(f"  Error (VQE vs Full FCI):  {error:.8f} Hartree")
    print("="*50)

if __name__ == "__main__":
    main()

O₂ Ground State Calculation for Basis: sto-3g
Using a manually constructed (8, 6) p-orbital active space

--- Running Full System Classical Calculation ---
Full System FCI (Exact) Energy: -147.74478939 Hartree
Classical calculations finished in 1.42s

--- Manually Constructing the Active Space Problem ---
Final problem (manual): (5, 3) electrons in 6 orbitals.
--------------------------------------------------

--- Running VQE on the 12-Qubit Problem ---


/tmp/ipykernel_1656841/3431311027.py:112: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  vqe_instance = VQE(Estimator(), ansatz, optimizer)


Number of qubits required: 12
VQE finished in 6998.42s


           Final Energy Summary for O₂
  VQE (12-Qubit) Energy: -34.95973961 Hartree
  FCI (Full System) Energy: -147.74478939 Hartree
--------------------------------------------------
  Error (VQE vs Full FCI):  112.78504978 Hartree


In [2]:
# Updated imports
from qiskit_ibm_runtime import QiskitRuntimeService, Estimator, Session
import numpy as np
import time
from datetime import datetime
from pyscf import gto, scf, fci
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import SLSQP

# ----- Main Calculation Function -----
def main():
    # ... (Classical calculations and Qiskit Nature problem setup)
    BASIS_SET = "3-21g"
    BOND_LENGTH = 0.8561
    r = BOND_LENGTH / np.sqrt(3)
    atom_str = (
        f"H {r:.4f} 0.0 0.0; "
        f"H {-r/2:.4f} {r * np.sqrt(3)/2:.4f} 0.0; "
        f"H {-r/2:.4f} {-r * np.sqrt(3)/2:.4f} 0.0"
    )
    mol = gto.M(atom=atom_str, basis=BASIS_SET, charge=1, spin=0, verbose=0)
    mf_uhf = scf.UHF(mol).run()
    mf_rhf = scf.RHF(mol).run()
    driver = PySCFDriver(atom=atom_str, basis=BASIS_SET, charge=1, spin=0)
    problem = driver.run()
    mapper = JordanWignerMapper()
    ansatz = UCCSD(
        problem.num_spatial_orbitals,
        problem.num_particles,
        mapper,
        initial_state=HartreeFock(
            problem.num_spatial_orbitals,
            problem.num_particles,
            mapper,
        ),
    )
    # ...

    # --- 2. Quantum Calculation (VQE) ---
    print("--- Running VQE on an IBM Quantum Device ---")
    start_time = time.time()

    # Initialize the runtime service
    service = QiskitRuntimeService()

    # Select the least-busy backend (with a min of 33 qubits for your problem)
    backend = service.least_busy(
        operational=True, simulator=False, min_num_qubits=33 # Note: The H3+ problem with 3-21g basis needs fewer qubits.
    )
    print(f"Using backend {backend.name}")

    # Run the VQE calculation within a Session context
    with Session(service=service, backend=backend) as session:
        # Create the Estimator instance from the qiskit_ibm_runtime package
        estimator = Estimator(session=session)

        # Define the VQE instance with the runtime Estimator
        vqe_instance = VQE(estimator, ansatz, SLSQP())
        vqe_instance.initial_point = np.zeros(ansatz.num_parameters)

        # Solve the ground state problem
        solver_vqe = GroundStateEigensolver(mapper, vqe_instance)
        result_vqe = solver_vqe.solve(problem)
        results = {} # Add this to store the result
        results['VQE'] = result_vqe.total_energies[0]
        print(f"Standard VQE Energy: {results['VQE']} Hartree")

    print(f"VQE finished in {time.time() - start_time:.2f}s\n")

ImportError: cannot import name 'BaseSampler' from 'qiskit.primitives' (/config/miniconda3/envs/vqe/lib/python3.11/site-packages/qiskit/primitives/__init__.py)

In [1]:
from qiskit_ibm_runtime import QiskitRuntimeService
import numpy as np
import time
from datetime import datetime

# --- Qiskit Runtime Primitives ---
from qiskit_ibm_runtime import Estimator, Session

# ----- Main Calculation Function -----
def main():
    # Load the account from the saved credentials
    service = QiskitRuntimeService()

    # Get the least-busy backend that is operational and has enough qubits
    backend = service.least_busy(
        operational=True, simulator=False, min_num_qubits=133
    )
    print(f"Using backend {backend.name}")

    # The rest of your VQE code goes here
    # ...

In [1]:
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit_nature.second_q.circuit.library import UCCSD
from qiskit_nature.second_q.mappers import QubitConverter

from qiskit.algorithms.optimizers import SLSQP
from qiskit.primitives import Estimator
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Options

# Define the H₂O molecule
driver = PySCFDriver(
    atom="O 0 0 0.115; H 0 0.754 -0.459; H 0 -0.754 -0.459",
    basis="sto-3g",
)

# Get the electronic structure problem
problem = driver.run()

# The number of qubits is determined by the number of spin orbitals in the basis set
num_qubits = problem.num_spin_orbitals
print(f"The H₂O problem requires {num_qubits} qubits.")

# Get the fermionic Hamiltonian
fermionic_hamiltonian = problem.hamiltonian.electronic_integrals

# Map the fermionic Hamiltonian to a qubit Hamiltonian using Jordan-Wigner mapping
mapper = JordanWignerMapper()
qubit_hamiltonian = mapper.map(fermionic_hamiltonian)
print("Mapped to qubit Hamiltonian.")

ImportError: cannot import name 'BaseEstimator' from 'qiskit.primitives' (/config/miniconda3/envs/end/lib/python3.13/site-packages/qiskit/primitives/__init__.py)

In [2]:
!pip list|grep qiskit

qiskit                  1.2.4
qiskit-aer              0.17.1
qiskit-algorithms       0.3.0
qiskit-nature           0.7.2


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from typing import Sequence

# Qiskit components
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp
from qiskit.primitives.base import BaseEstimatorV2
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

# Qiskit Nature for molecular simulation
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer

# Qiskit IBM Runtime for running on hardware
from qiskit_ibm_runtime import QiskitRuntimeService, Estimator

# ====================================================================
# 1. DEFINE THE MOLECULAR PROBLEM FOR LiH
# ====================================================================
# Using a standard bond length for LiH (1.595 angstroms)
molecule_string = "Li 0 0 0; H 0 0 1.595"
driver = PySCFDriver(atom=molecule_string, basis="sto3g")
problem = driver.run()

# --- FREEZE CORE / DEFINE ACTIVE SPACE ---
# LiH has 4 electrons. The 1s orbital of Li is the core.
# We freeze the core (2 electrons in 1 orbital) and solve for the remaining
# 2 chemically active electrons in a space of 3 active orbitals.
# FIX: Correcting the keyword argument for orbitals.
transformer = ActiveSpaceTransformer(num_electrons=2, num_spatial_orbitals=3)
problem = transformer.transform(problem)

# Mapper
mapper = JordanWignerMapper()
qubit_op = mapper.map(problem.hamiltonian.second_q_op())
num_qubits = qubit_op.num_qubits

# --- CLASSICAL EXACT CALCULATION (FCI) FOR COMPARISON ---
# Note: This classical FCI is for the full molecule, not just the active space.
# It serves as a benchmark for the overall ground state energy.
from pyscf import gto, scf, fci
mol = gto.Mole()
mol.build(atom=molecule_string, basis="sto3g")
mf = scf.RHF(mol).run()
cisolver = fci.FCI(mol, mf.mo_coeff)
fci_energy = cisolver.kernel()[0]
fci_result = fci_energy + mol.energy_nuc()
print(f"Classical FCI (exact) energy for LiH: {fci_result:.12f}")

# ====================================================================
# 2. SELECT BACKEND AND PREPARE CIRCUITS
# ====================================================================
service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, min_num_qubits=num_qubits, simulator=False
)
print(f"Using backend: {backend.name}")

# Ansatz (UCCSD) using the active space problem definition
initial_state = HartreeFock(
    problem.num_spatial_orbitals,
    problem.num_particles,
    qubit_mapper=mapper
)
ansatz = UCCSD(
    num_spatial_orbitals=problem.num_spatial_orbitals,
    num_particles=problem.num_particles,
    initial_state=initial_state,
    qubit_mapper=mapper
)

# Transpile
print("Transpiling circuit...")
pm = generate_preset_pass_manager(optimization_level=3, backend=backend)
ansatz_ibm = pm.run(ansatz)

# Observable with layout
observable_ibm = qubit_op.apply_layout(ansatz_ibm.layout)
print(f"Ansatz has {ansatz_ibm.num_parameters} parameters.")

# ====================================================================
# 3. DEFINE COST AND CALLBACK FUNCTIONS
# ====================================================================
def cost_func(
    params: Sequence,
    ansatz: QuantumCircuit,
    hamiltonian: SparsePauliOp,
    estimator: BaseEstimatorV2,
) -> float:
    """Ground state energy evaluation."""
    pub = (ansatz, hamiltonian, [params])
    result = estimator.run([pub]).result()[0]
    return result.data.evs.real.item()

def build_callback(
    ansatz: QuantumCircuit,
    hamiltonian: SparsePauliOp,
    estimator: BaseEstimatorV2,
    callback_dict: dict,
):
    def callback(current_vector):
        callback_dict["iters"] += 1
        callback_dict["prev_vector"] = current_vector

        pub = (ansatz, hamiltonian, [current_vector])
        result = estimator.run([pub]).result()[0]
        current_cost = result.data.evs.real.item()

        callback_dict["cost_history"].append(current_cost)
        print(
            f"Iters: {callback_dict['iters']:3d}  |  Energy: {current_cost:.12f}",
            end="\r",
            flush=True,
        )
    return callback

# ====================================================================
# 4. RUN THE VQE OPTIMIZATION
# ====================================================================
num_params = ansatz_ibm.num_parameters
initial_params = 2 * np.pi * np.random.random(num_params)

callback_dict = {
    "prev_vector": None,
    "iters": 0,
    "cost_history": [],
}

print("\nStarting VQE run for LiH on real hardware. This may take a while...")

# Instantiate the Estimator for job mode
options = {
    "resilience_level": 1,
    "default_shots": 10000
}
estimator = Estimator(mode=backend, options=options)


callback = build_callback(
    ansatz_ibm, observable_ibm, estimator, callback_dict
)

# Increased maxiter for a more complex problem
res = minimize(
    cost_func,
    x0=initial_params,
    args=(ansatz_ibm, observable_ibm, estimator),
    callback=callback,
    method="cobyla",
    options={"maxiter": 150},
)

print("\nOptimization complete.")

# ====================================================================
# 5. VISUALIZE AND PRINT RESULTS
# ====================================================================
# The VQE energy is for the active space, so we add the frozen energy back
# to compare with the full FCI result.
vqe_energy_active_space = res.fun
total_vqe_energy = vqe_energy_active_space + problem.energy_shift['FrozenCore']
print(f"\nEstimated active space energy: {vqe_energy_active_space:.12f}")
print(f"Total VQE ground state energy: {total_vqe_energy:.12f}")
print(f"Classical FCI (exact) energy:    {fci_result:.12f}")
error = abs(total_vqe_energy - fci_result)
print(f"Absolute error: {error:.6f} Hartrees")

# Plotting the convergence of the active space energy
plt.figure(figsize=(10, 6))
plt.plot(callback_dict["cost_history"], lw=2, label="VQE Energy (Active Space)")
# To compare on the plot, we need the FCI energy of just the active space
fci_active_space_energy = fci_result - problem.energy_shift['FrozenCore']
plt.axhline(y=fci_active_space_energy, color='r', linestyle='--', label='FCI Energy (Active Space)')
plt.xlabel("Iteration")
plt.ylabel("Energy (Hartrees)")
plt.title(f"VQE Convergence for LiH Molecule on {backend.name}")
plt.legend()
plt.grid(True)
plt.show()



converged SCF energy = -7.86202386012712
Classical FCI (exact) energy for LiH: -6.887084294196
Using backend: ibm_torino
Transpiling circuit...
Ansatz has 8 parameters.

Starting VQE run for LiH on real hardware. This may take a while...
